In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('/kaggle/input/cardiovascular-diseases-risk-prediction-dataset/CVD_cleaned.csv')

In [ ]:
df.shape

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
df.columns

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
for i in df.select_dtypes(include='object'):
    
    print(df[i].value_counts(normalize=True))

In [ ]:
df.loc[(df['Checkup']=='Never')& (df['Heart_Disease']=='Yes')]

In [ ]:
df['Diabetes'].value_counts(normalize=True)

In [ ]:
df.loc[(df['Diabetes']=='NO') | (df['Diabetes']=='No, pre-diabetes or borderline diabetes')]

In [ ]:
df['Diabetes'].replace('No, pre-diabetes or borderline diabetes','No',inplace=True)

In [ ]:
df['Diabetes'].replace('Yes, but female told only during pregnancy','No',inplace=True)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df['Checkup'].value_counts()

In [ ]:
od = OrdinalEncoder(categories=[['Within the past year','Within the past 2 years','Within the past 5 years','5 or more years ago','Never']])

In [ ]:
df['Checkup']=od.fit_transform(X=df[['Checkup']])

In [ ]:
df['Checkup']

In [ ]:
df.head(5)

In [ ]:
df['General_Health'].value_counts()

In [ ]:
od = OrdinalEncoder(categories=[['Very Good','Good','Excellent','Fair','Poor']])
df['General_Health']=od.fit_transform(X=df[['General_Health']])

In [ ]:
df['Age_Category'].value_counts()

In [ ]:
od = OrdinalEncoder(categories=[['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80+']])
df['Age_Category']=od.fit_transform(X=df[['Age_Category']])

In [ ]:
df['Age_Category'].value_counts()

In [ ]:
df.describe()

In [ ]:
df=pd.get_dummies(df,drop_first=True)

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df= df.drop(columns=['Height_(cm)','Weight_(kg)'])

In [ ]:
df.head(5)

In [ ]:
import statsmodels.api as sm
x=df.drop(columns='Heart_Disease_Yes')
y=df['Heart_Disease_Yes']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.7)
x_c=sm.add_constant(xtrain)
model1=sm.Logit(ytrain,x_c).fit()
model1.summary()

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
modellr=lr.fit(xtrain,ytrain)
pred_train=lr.predict(xtrain)
pred_test=lr.predict(xtest)
from sklearn.metrics import roc_curve,roc_auc_score,accuracy_score,classification_report
print("roc_auc_score",roc_auc_score(ytest,pred_test))
print("auccracy_score",accuracy_score(ytest,pred_test))
print(classification_report(ytest,pred_test))

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
g=GaussianNB()
model_nb=g.fit(xtrain,ytrain)
pred_test=g.predict(xtest)
print(classification_report(ytest,pred_test))

In [ ]:
print(accuracy_score(ytest,pred_test))

In [ ]:
print(roc_auc_score(ytest,pred_test))

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb=BernoulliNB()
bnb_model=bnb.fit(xtrain,ytrain)
pred_test=bnb.predict(xtest)
print(accuracy_score(ytest,pred_test))
print(classification_report(ytest,pred_test))
print(roc_auc_score(ytest,pred_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=10)
model_dt=dt.fit(xtrain,ytrain)
pred_test=dt.predict(xtest)
print(accuracy_score(ytest,pred_test))
print(roc_auc_score(ytest,pred_test))
print(classification_report(ytest,pred_test))

In [ ]:
dt.feature_importances_

In [ ]:
fea = pd.DataFrame(data=dt.feature_importances_ , index = xtrain.columns,columns=['Features_imp'])

In [ ]:
fea.sort_values(by=['Features_imp'],ascending=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
turned_para = [{'criterion':['entropy','gini'],
               'max_depth':[2,3,5,6,7,8,9,10]}]

In [ ]:
grid = GridSearchCV(estimator=dt,param_grid=turned_para,cv=5,verbose=2)

In [ ]:
tree_grid_model = grid.fit(xtrain,ytrain)
print('Best parameter for decision tree classifier: ', tree_grid_model.best_params_)

In [ ]:
pred_t=tree_grid_model.predict(xtest)

In [ ]:
print(accuracy_score(ytest,pred_t))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf=RandomForestClassifier(random_state=10)
rf.fit(xtrain,ytrain)
pred_rf_train = rf.predict(xtrain)

In [ ]:
accuracy_score(ytrain,pred_rf_train)

In [ ]:
pred_rf_test = rf.predict(xtest)
accuracy_score(ytest,pred_rf_test)

In [ ]:
param=[{
 'criterion': ['gini'],
 'max_features': ['sqrt'],
 'max_leaf_nodes': [2,3],
 'min_samples_leaf': [2,4],
 'min_samples_split': [2,4],
 'n_estimators': [250]}]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xsc=sc.fit_transform(x)

In [ ]:
xsc = pd.DataFrame(data=xsc,columns=x.columns)
xsc

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=10)

In [ ]:
grid = GridSearchCV(estimator=RandomForestClassifier(random_state=10),param_grid=param,cv=5,verbose=2)
tree_grid_model = grid.fit(xtrain,ytrain)
print('Best parameter for decision tree classifier: ', tree_grid_model.best_params_)

In [ ]:
pred = grid.predict(xtest)
accuracy_score(ytest,pred1)

In [ ]:
pred1=tree_grid_model.predict(xtest)

In [ ]:
print(classification_report(ytest,pred1))